In [3]:
#bootstrap módszerrel V p-értékének meghatározása
import pandas as pd
import numpy as np

dk = pd.read_csv('breast_cancer_mod2.csv')

#bootstrap beállítások
B = 30  #ismétlések száma
V_bootstrap = []

def calculate_Vm(data):
    D = data[data['status'] == 1]['rfstime'].max()
    n_1 = int(len(data[(data['hormon'] == 1)])) #hormonkezelt betegek száma
    n_2 = int(len(data[(data['hormon'] == 0)])) #nem hormonkezelt betegek száma
    n = n_1 + n_2 #összes beteg száma

    #cenzorálás szerint adatok különválasztása
    filtered1_data = data[data['status'] == 1]
    filtered2_data = data[data['status'] == 0] #cenzorált rész

    #segédtáblázat létrehozása
    time_range = range(1, D+1)
    result_data = pd.DataFrame({'idő': time_range})

    #kezdeti értékek
    S_t = 1
    L1_t = 1
    L2_t = 1

    #ciklus minden időpontra
    for t in time_range:
        Y_i = int(len(filtered1_data[filtered1_data['rfstime'] >= t])) #összes vizsgált beteg az adott időpontban
        d_i = int(len(filtered1_data[filtered1_data['rfstime'] == t])) #összes esemény az adott időpontban

        #S becslése
        term_1 = 1 - (d_i / Y_i) if Y_i > 0 else np.nan
        S_t_prev = S_t  #előző S érték tárolása
        S_t = S_t * term_1 if not np.isnan(term_1) else S_t_prev  #ha term_3 nan, marad az előző érték

        #Delta S(t) kiszámítása
        Delta_S_t = S_t - S_t_prev

        Y_i1 = int(len(filtered2_data[(filtered2_data['hormon'] == 1) & (filtered2_data['rfstime'] >= t)])) #hormonkezeltek száma cenzorált adatokban
        Y_i2 = int(len(filtered2_data[(filtered2_data['hormon'] == 0) & (filtered2_data['rfstime'] >= t)])) #nem hormonkezeltek cenzorált adatokban
        d_i1 = int(len(filtered2_data[(filtered2_data['hormon'] == 1) & (filtered2_data['rfstime'] == t)])) #halálok cenzorált adatoknál
        d_i2 = int(len(filtered2_data[(filtered2_data['hormon'] == 0) & (filtered2_data['rfstime'] == t)]))

        #L_1, L_2 becslése
        term_2 = 1 - (d_i1 / Y_i1) if Y_i1 > 0 else np.nan
        term_3 = 1 - (d_i2 / Y_i2) if Y_i2 > 0 else np.nan

        L1_t_prev = L1_t
        L2_t_prev = L2_t

        L1_t = L1_t_prev * term_2 if not np.isnan(term_2) else L1_t_prev
        L2_t = L2_t_prev * term_3 if not np.isnan(term_3) else L2_t_prev
        #értékek hozzáadása az eredménytáblázathoz
        result_data.loc[t-1, 'Y_i'] = Y_i
        result_data.loc[t-1, 'd_i'] = d_i
        result_data.loc[t-1, '1-(d_i/Y_i)'] = term_1
        result_data.loc[t-1, 'S(t)'] = S_t
        result_data.loc[t-1, 'Delta S(t)'] = Delta_S_t
        result_data.loc[t-1, 'L1(t)'] = L1_t
        result_data.loc[t-1, 'L2(t)'] = L2_t

    #V_m kiszámítása
    #segédtáblázat létrehozása
    time_range = range(1, D+1)

    result_data1 = pd.DataFrame({'idő': time_range})



    #ciklus minden időpontra
    for t in time_range:
        #hormonkezeltek
        Y_i1 = int(len(data[(data['hormon'] == 1) & (data['rfstime'] >= t)]))  #vizsgált hormonkezeltek száma
        d_i1 = int(len(data[(data['hormon'] == 1) & (data['rfstime'] == t) & (data['status'] == 1)]))  #meghalt/kiújult rák

        #nem hormonkezeltek
        Y_i2 = int(len(data[(data['hormon'] == 0) & (data['rfstime'] >= t)]))  #vizsgált nem hormonkezeltek száma
        d_i2 = int(len(data[(data['hormon'] == 0) & (data['rfstime'] == t) & (data['status'] == 1)]))  #meghalt/kiújult rák

        #összes vizsgált egyed és összes kiújult rák/meghalt
        Y_i = Y_i1 + Y_i2
        d_i = d_i1 + d_i2


        if Y_i > 0: #ne legyen 0 az osztó
            term_1 = Y_i1 * (d_i / Y_i) if Y_i > 0 else 0
            term_2 = d_i1 - term_1
            term_3 = (Y_i1 / Y_i) * (Y_i2 / Y_i) * ((Y_i - d_i) / (Y_i - 1)) * d_i if Y_i > 1 else 0
        else:
            term_1 = term_2 = term_3 = np.nan  #ha Y_i nulla, a kifejezések nincsenek értelmezve

        #hozzáadjuk az értékeket az eredménytáblához
        result_data1.loc[t-1, 'Y_i1'] = Y_i1
        result_data1.loc[t-1, 'd_i1'] = d_i1
        result_data1.loc[t-1, 'Y_i2'] = Y_i2
        result_data1.loc[t-1, 'd_i2'] = d_i2
        result_data1.loc[t-1, 'Y_i'] = Y_i
        result_data1.loc[t-1, 'd_i'] = d_i
        result_data1.loc[t-1, 'Y_i1*(d_i/Y_i)'] = term_1
        result_data1.loc[t-1, 'd_i1-Y_i1*(d_i/Y_i)'] = term_2
        result_data1.loc[t-1, '(Y_i1/Y_i)*(Y_i2/Y_i))*((Y_i-d_i)/(Y_i-1))*d_i'] = term_3

    #V_m-ek szuprémuma

    #Keresett intervallum
    m_values = range(int(D*0.05), D - int(D*0.05) + 1)
    best_Vm = -np.inf
    best_m = None

    for m in m_values:
        #c_m számítása
        numerator = result_data.loc[:m, 'L1(t)'] * result_data.loc[:m, 'L2(t)'] * result_data.loc[:m, 'Delta S(t)']
        numerator = numerator / (((n_1/n) * result_data.loc[:m, 'L1(t)']) + ((n_2/n) * result_data.loc[:m, 'L2(t)']))

        denominator = result_data.loc[m:, 'L1(t)'] * result_data.loc[m:, 'L2(t)'] * result_data.loc[m:, 'Delta S(t)']
        denominator = denominator / (((n_1/n) * result_data.loc[m:, 'L1(t)']) + ((n_2/n) * result_data.loc[m:, 'L2(t)']))

        c_m = numerator.sum() / denominator.sum() if denominator.sum() != 0 else np.nan

        #w_i2 beállítása
        result_data1['w_i2'] = -1
        result_data1['w_i2'] = result_data1['w_i2'].astype(float)
        result_data1.loc[m:, 'w_i2'] = c_m

        #V_m számlálója
        result_data1['first_part'] = result_data1['w_i2'] * (result_data1['d_i1-Y_i1*(d_i/Y_i)'])

        #V_m nevezője
        result_data1['second_part'] = result_data1['w_i2']**2 * (result_data1['(Y_i1/Y_i)*(Y_i2/Y_i))*((Y_i-d_i)/(Y_i-1))*d_i'])

        sum_first_part = result_data1['first_part'].sum()
        sum_second_part = result_data1['second_part'].sum()

        V_m = sum_first_part / np.sqrt(sum_second_part) if sum_second_part > 0 else np.nan

        if not np.isnan(V_m) and V_m > best_Vm:
            best_Vm = V_m
            best_m = m

    return best_Vm

#eredeti adatokkal számított V
observed_V = calculate_Vm(dk)
print(observed_V)

#bootstrap iterációk
for _ in range(B):
    dk_bootstrap = dk.sample(frac=0.5, replace=True)  #visszatevéses mintavételezés -50%-os minta
    V_boot = calculate_Vm(dk_bootstrap)  #új V számítása a bootstrap mintán
    V_bootstrap.append(V_boot)

#bootstrap eloszlás numpy tömbbé alakítása
V_bootstrap = np.array(V_bootstrap)
print(V_bootstrap)

#kritikus érték meghatározása (0,0253 szignifikanciaszint)
V_crit = np.percentile(V_bootstrap, 98.735)

#p-érték számítása (hányszor nagyobb a V_boot, mint az eredeti/30)
p_value = np.sum(V_bootstrap >= V_observed)/30

print(f"V kritikus értéke (98,735%-os szint): {V_crit}")
print(f"p-érték: {p_value}")


4.66190268222463
[3.72480466 4.13786242 5.09089155 2.68275903 4.49416236 4.87590239
 2.92044446 3.86453613 2.34146923 3.15923069 2.35160122 5.08881021
 3.15905351 3.33677941 3.7422567  3.76592276 2.96035429 4.18608474
 3.45563138 3.15777943 2.27286331 3.78372952 3.18201776 4.73552387
 5.11165515 4.1624058  4.74687164 2.91875799 3.58567846 3.84005215]
V kritikus értéke (98,735%-os szint): 5.104038022682863
p-érték: 0.2


In [2]:
import numpy as np

# Példa adatok
V_boot = np.array([2.1, 3.5, 4.0, 5.6, 6.1, 7.2])
V_observed = 4.5

# Logikai feltétel teljesülésének számolása
count = np.sum(V_boot >= V_observed)

print(f"Elemszám: {count}")

Elemszám: 3
